# Homework: LLM Orchestration and Ingestion

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Our FAQ documents change with time: students add more records
and edit existing ones. We need to keep our index in sync.

There are two ways of doing it:

1. Incremental: you only update records that got changed, created or deleted
2. Full update: you recreate the entire index from scratch

In this homework, we'll look at full update. We will run our 
indexing pipeline daily and re-create the index from scracth 
each time we run. 


For that, we created two FAQ documents for LLM Zoomcamp

* [version 1](https://docs.google.com/document/d/1qZjwHkvP0lXHiE4zdbWyUXSVfmVGzougDD6N37bat3E/edit)
* [version 2](https://docs.google.com/document/d/1T3MdwUvqCL3jrh3d3VCXQ8xE0UqRzI3bfgpfBq3ZWG0/edit)

First, we will run our ingestion pipeline with version 1 
and then with version 2.

# Q1. Running Mage

Clone the same repo we used in the module and run mage:


```bash
git clone https://github.com/mage-ai/rag-project
```

Add the following libraries to the requirements document:

```
python-docx
elasticsearch
```

Make sure you use the latest version of mage:

```bash
docker pull mageai/mageai:llm
```

Start it:

```bash
./scripts/start.sh
```

Now mage is running on [http://localhost:6789/](http://localhost:6789/)

What's the version of mage? 

## Answer

The version of Mage running is `v0.9.72`

# Creating a RAG pipeline

Create a RAG pipeline

# Q2. Reading the documents

Now we can ingest the documents. Create a custom code ingestion
block 

Let's read the documents. We will use the same code we used
for parsing FAQ: [parse-faq-llm.ipynb](parse-faq-llm.ipynb)


Use the following document_id: 1qZjwHkvP0lXHiE4zdbWyUXSVfmVGzougDD6N37bat3E

Which is the document ID of
[LLM FAQ version 1](https://docs.google.com/document/d/1qZjwHkvP0lXHiE4zdbWyUXSVfmVGzougDD6N37bat3E/edit)

Copy the code to the editor
How many FAQ documents we processed?

* 1
* 2
* 3
* 4


## Answer

We have processed `1` FAQ document

# Q3. Chunking

We don't really need to do any chuncking because our documents
already have well-specified boundaries. So we just need
to return the documents without any changes.

So let's go to the transformation part and add a custom code
chunking block:

```python
documents = []

for doc in data['documents']:
    doc['course'] = data['course']
    # previously we used just "id" for document ID
    doc['document_id'] = generate_document_id(doc)
    documents.append(doc)

print(len(documents))

return documents
```


Where `data` is the input parameter to the transformer.

And the `generate_document_id` is defined in the same way
as in module 4:

```python
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id
```

Note: if instead of a single dictionary you get a list, 
add a for loop:

```python
for course_dict in data:
    ...
```

You can check the type of `data` with this code:

```python
print(type(data))
```

How many documents (chunks) do we have in the output?

* 66
* 76
* 86
* 96

## Answer

We have `86` documents in the output

# Tokenization and embeddings

We don't need any tokenization, so we skip it.

Because currently it's required in mage, we can create 
a dummy code block:

* Create a custom code block
* Don't change it

Because we will use text search, we also don't need embeddings,
so skip it too.

If you want to use sentence transformers - the ones from module
3 - you don't need tokenization, but need embeddings
(you don't need it for this homework)

# Q4. Export

Now we're ready to index the data with elasticsearch. For that,
we use the Export part of the pipeline

* Go to the Export part
* Select vector databases -> Elasticsearch
* Open the code for editing

Because we won't use vector search, but usual text search, we
will need to adjust the code.

First, let's change the line where we read the index name:

```python
index_name = kwargs.get('index_name', 'documents')
``` 

To `index_name_prefix` - we will parametrize it with the day
and time we run the pipeline

```python
from datetime import datetime

index_name_prefix = kwargs.get('index_name', 'documents')
current_time = datetime.now().strftime("%Y%m%d_%M%S")
index_name = f"{index_name_prefix}_{current_time}"
print("index name:", index_name)
```


We will need to save the name in a global variable, so it can be accessible in other code blocks

```python
from mage_ai.data_preparation.variable_manager import set_global_variable

set_global_variable('hallowed_automaton', 'index_name', index_name)
```

Where your pipeline name is the name of the pipeline, e.g.
`transcendent_nexus` (replace the space with underscore `_`)



Replace index settings with the settings we used previously:

```python
index_settings = {
    "settings": {
        "number_of_shards": number_of_shards,
        "number_of_replicas": number_of_replicas
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "document_id": {"type": "keyword"}
        }
    }
}
```

Remove the embeddings line:

```python
if isinstance(document[vector_column_name], np.ndarray):
    document[vector_column_name] = document[vector_column_name].tolist()
```
 
At the end (outside of the indexing for loop), print the last document:

```python
        print(document)
```

Now execute the block.

What's the last document id?

Also note the index name.

## Answer

The last document id is `d8c4c7bb` (index name was `documents_20240815_4550`)

# Q5. Testing the retrieval

Now let's test the retrieval. Use mage or jupyter notebook to
test it.

Let's use the following query: "When is the next cohort?"

What's the ID of the top matching result?

## Answer

The ID of the top matching result is `bf024675`

# Q6. Reindexing

Our FAQ document changes: every day course participants add
new records or improve existing ones.

Imagine some time passed and the document changed. For that we have another version of the FAQ document: [version 2](https://docs.google.com/document/d/1T3MdwUvqCL3jrh3d3VCXQ8xE0UqRzI3bfgpfBq3ZWG0/edit).

The ID of this document is `1T3MdwUvqCL3jrh3d3VCXQ8xE0UqRzI3bfgpfBq3ZWG0`.

Let's re-execute the entire pipeline with the updated data.

For the same query "When is the next cohort?". What's the ID of the top matching result?


## Answer

The ID of the top matching result is `b6fa77f3`

# Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw5
* It's possible that your answers won't match exactly. If it's the case, select the closest one.

# Annex

A simple script to do the exercises without Mage

```python
from elasticsearch import Elasticsearch
import os
import io
import json
import requests
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import docx
from datetime import datetime


def clean_line(line):
    line = line.strip()
    line = line.strip("\uFEFF")
    return line


def read_faq(file_id):
    url = f"https://docs.google.com/document/d/{file_id}/export?format=docx"

    response = requests.get(url)
    response.raise_for_status()

    with io.BytesIO(response.content) as f_in:
        doc = docx.Document(f_in)

    questions = []

    question_heading_style = "heading 2"
    section_heading_style = "heading 1"

    heading_id = ""
    section_title = ""
    question_title = ""
    answer_text_so_far = ""

    for p in doc.paragraphs:
        style = p.style.name.lower()
        p_text = clean_line(p.text)

        if len(p_text) == 0:
            continue

        if style == section_heading_style:
            section_title = p_text
            continue

        if style == question_heading_style:
            answer_text_so_far = answer_text_so_far.strip()
            if (
                answer_text_so_far != ""
                and section_title != ""
                and question_title != ""
            ):
                questions.append(
                    {
                        "text": answer_text_so_far,
                        "section": section_title,
                        "question": question_title,
                    }
                )
                answer_text_so_far = ""

            question_title = p_text
            continue

        answer_text_so_far += "\n" + p_text

    answer_text_so_far = answer_text_so_far.strip()
    if answer_text_so_far != "" and section_title != "" and question_title != "":
        questions.append(
            {
                "text": answer_text_so_far,
                "section": section_title,
                "question": question_title,
            }
        )

    return questions


def load_documents(faq_documents):
    documents = []

    for course, file_id in faq_documents.items():
        print(course)
        course_documents = read_faq(file_id)
        documents.append({"course": course, "documents": course_documents})

    return documents


def generate_document_id(doc):
    import hashlib

    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id


def chunk_documents(data):
    documents = []

    for doc in data["documents"]:
        doc["course"] = data["course"]
        # previously we used just "id" for document ID
        doc["document_id"] = generate_document_id(doc)
        documents.append(doc)

    print(len(documents))

    return documents


def main():
    documents = load_documents(faq_documents=faq_documents_end)

    # Chunk documents
    chunked_documents = chunk_documents(data=documents[0])

    # Open connection to elasticsearch
    es_client = Elasticsearch("http://localhost:9200")
    es_client.info()

    index_name_prefix = "documents"
    current_time = datetime.now().strftime("%Y%m%d_%M%S")
    index_name = f"{index_name_prefix}_{current_time}"
    print("index name:", index_name)

    number_of_shards = 1
    number_of_replicas = 0
    vector_column_name = "embedding"
    dimensions = []

    index_settings = {
        "settings": {"number_of_shards": number_of_shards, "number_of_replicas": 0},
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"},
                "document_id": {"type": "keyword"},
            }
        },
    }

    if not es_client.indices.exists(index=index_name):
        es_client.indices.create(index=index_name)
        print("Index created with properties:", index_settings)
        print("Embedding dimensions:", dimensions)

    print(
        f"Indexing {len(chunked_documents)} documents to Elasticsearch index {index_name}"
    )
    for document in chunked_documents:
        print(f'Indexing document {document["document_id"]}')

        es_client.index(index=index_name, document=document)
        print(document)

    # Retrieval
    query = "When is the next cohort?"

    # Search for the question of Q1 using elasticsearch
    def elastic_search(query_text: str, index_name: str):
        search_query = {
            "size": 2,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["question", "text"],
                            "type": "best_fields",
                        }
                    },
                }
            },
        }

        es_results = es_client.search(index=index_name, body=search_query)

        result_docs = []

        for hit in es_results["hits"]["hits"]:
            result_docs.append(hit["_source"])

        return result_docs

    search_results = elastic_search(query_text=query, index_name=index_name)
    return search_results


if __name__ == "__main__":
    faq_documents_ini = {
        "llm-zoomcamp": "1m2KexowAXTmexfC5rVTCSnaShvdUQ8Ag2IEiwBDHxN0",
    }

    faq_documents_end = {
        "llm-zoomcamp": "1T3MdwUvqCL3jrh3d3VCXQ8xE0UqRzI3bfgpfBq3ZWG0",
    }

    search_results_ini = main(faq_documents=faq_documents_ini)
    search_results_end = main(faq_documents=faq_documents_end)
```